In [79]:
%matplotlib inline
import os

In [80]:
# Load the dataset
originals = open('dataset/lns_comments.txt', 'r').readlines()
corpus = open('dataset/lns_stemmed.txt', 'r').readlines()
labels = open('dataset/lns_labels.txt', 'r').readlines()
labels = [int(float(x)) for x in labels]

In [81]:
import bald_latin

corpus, labels = bald_latin.remove_cyrillic_comments(corpus, labels)
originals, _ = bald_latin.remove_cyrillic_comments(originals, range(len(originals)))
corpus = bald_latin.remove_serbian_accents(corpus)

Cyrillic comments make up 1.24624624625 percent
Cyrillic comments make up 1.24643339841 percent


In [82]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

croatian_stop_words = set([u"a",u"ako",u"ali",u"bi",u"bih",u"bila",u"bili",u"bilo",u"bio",u"bismo",u"biste",u"biti",u"bumo",u"da",u"do",u"duž",u"ga",u"hoće",u"hoćemo",u"hoćete",u"hoćeš",u"hoću",u"i",u"iako",u"ih",u"ili",u"iz",u"ja",u"je",u"jedna",u"jedne",u"jedno",u"jer",u"jesam",u"jesi",u"jesmo",u"jest",u"jeste",u"jesu",u"jim",u"joj",u"još",u"ju",u"kada",u"kako",u"kao",u"koja",u"koje",u"koji",u"kojima",u"koju",u"kroz",u"li",u"me",u"mene",u"meni",u"mi",u"mimo",u"moj",u"moja",u"moje",u"mu",u"na",u"nad",u"nakon",u"nam",u"nama",u"nas",u"naš",u"naša",u"naše",u"našeg",u"ne",u"nego",u"neka",u"neki",u"nekog",u"neku",u"nema",u"netko",u"neće",u"nećemo",u"nećete",u"nećeš",u"neću",u"nešto",u"ni",u"nije",u"nikoga",u"nikoje",u"nikoju",u"nisam",u"nisi",u"nismo",u"niste",u"nisu",u"njega",u"njegov",u"njegova",u"njegovo",u"njemu",u"njezin",u"njezina",u"njezino",u"njih",u"njihov",u"njihova",u"njihovo",u"njim",u"njima",u"njoj",u"nju",u"no",u"o",u"od",u"odmah",u"on",u"ona",u"oni",u"ono",u"ova",u"pa",u"pak",u"po",u"pod",u"pored",u"prije",u"s",u"sa",u"sam",u"samo",u"se",u"sebe",u"sebi",u"si",u"smo",u"ste",u"su",u"sve",u"svi",u"svog",u"svoj",u"svoja",u"svoje",u"svom",u"ta",u"tada",u"taj",u"tako",u"te",u"tebe",u"tebi",u"ti",u"to",u"toj",u"tome",u"tu",u"tvoj",u"tvoja",u"tvoje",u"u",u"uz",u"vam",u"vama",u"vas",u"vaš",u"vaša",u"vaše",u"već",u"vi",u"vrlo",u"za",u"zar",u"će",u"ćemo",u"ćete",u"ćeš",u"ću",u"što"])


vectorizer = TfidfVectorizer(
    strip_accents="unicode",
    lowercase=True,
    ngram_range=(1,2),
    min_df=2,
    norm='l2',
    smooth_idf=True,
    use_idf=True,
    stop_words=croatian_stop_words)

X = vectorizer.fit_transform(corpus)

In [83]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

distance = (X * X.T).todense()
distance = distance.A

In [178]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=20,algorithm='brute', metric='cosine')
mv = 5000
neigh.fit(X[:mv], labels[:mv])
proba = neigh.predict_proba(X[mv:])

In [7]:
def knn_classify(distance, labels, k=20):
    classes = []
    for idx, row in enumerate(distance):
        knbrs = np.argsort(row)[-2:-2-k:-1]
        
        label = 0
        for nbr in knbrs: 
            label += labels[nbr]
            
        classes.append(label/len(knbrs))
        
    return classes

predictions = knn_classify(distance, labels)

concensus = 0.85
error = 0
count = 0
for label, pred in zip(labels, predictions):
    if pred >= concensus or pred <= 1 - concensus:
        count += 1
        if np.round(pred) != label:
            error += 1
        
    else:
        continue

print count, error, str(error / (count + 0.0) * 100) + "%"

636 69 10.8490566038%


In [113]:
from sklearn.naive_bayes import MultinomialNB

mv = 5000

clf = MultinomialNB().fit(X[:mv], np.array(labels[:mv], np.uint8))

pr = clf.predict(X[mv:])
proba = clf.predict_proba(X[mv:])


In [10]:
from sklearn.linear_model import SGDClassifier

sgd = SGDClassifier(penalty='l1', alpha=1e-4, n_iter=20, random_state=210)
sgd.fit(X[:mv], np.array(labels[:mv], np.uint8))
pr = sgd.predict(X[mv:])
print pr

np.mean(np.array(labels[mv:], np.uint8) == pr)

[0 0 1 ..., 1 0 1]


0.70050761421319796

In [299]:

from sklearn.svm import SVR
svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(X[:mv], np.array(labels[:mv], np.uint8))
pr = svr_rbf.predict(X[mv:])


In [303]:
print pr

sum = 0
count = 0
for p, l in zip(pr, labels[mv:]): 
    if p <= 0.0 or p >= 1:
        count += 1
        if round(p) != round(l):
            sum += 1
            
print count, sum

[ 0.36987503  0.07626233  0.58753726 ...,  1.24794644 -0.13420733
  0.59776429]
232 42
